In [88]:
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords, wordnet
from nltk.tokenize import TweetTokenizer, RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.decomposition import PCA

In [2]:
sentiments = pd.read_csv('./data/judge-1377884607_tweet_product_company.csv',encoding='ISO-8859-1')
sentiments.tail()

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product
9088,Ipad everywhere. #SXSW {link},iPad,Positive emotion
9089,"Wave, buzz... RT @mention We interrupt your re...",NaN,No emotion toward brand or product
9090,"Google's Zeiger, a physician never reported po...",NaN,No emotion toward brand or product
9091,Some Verizon iPhone customers complained their...,NaN,No emotion toward brand or product
9092,Ï¡Ïàü_ÊÎÒ£Áââ_£â_ÛâRT @...,NaN,No emotion toward brand or product


In [3]:
sentiments['is_there_an_emotion_directed_at_a_brand_or_product'].value_counts()

No emotion toward brand or product    5389
Positive emotion                      2978
Negative emotion                       570
I can't tell                           156
Name: is_there_an_emotion_directed_at_a_brand_or_product, dtype: int64

In [4]:
sentiments.dropna(subset=['tweet_text'],inplace=True)

In [114]:
def tokenize(text):
    sw = stopwords.words('english')
    text = [str(phrase).lower() for phrase in text]
    # tokenizer_ = TweetTokenizer(preserve_case=False)
    tokenizer_ = RegexpTokenizer(r"([a-zA-Z]+(?:’[a-z]+)?)")
    tokens = [' '.join(tokenizer_.tokenize(str(doc))) for doc in text]
    return list(tokens)

In [38]:
TfidfTransformer?

Init signature:
TfidfTransformer(
    *,
    norm='l2',
    use_idf=True,
    smooth_idf=True,
    sublinear_tf=False,
)
Docstring:     
Transform a count matrix to a normalized tf or tf-idf representation

Tf means term-frequency while tf-idf means term-frequency times inverse
document-frequency. This is a common term weighting scheme in information
retrieval, that has also found good use in document classification.

The goal of using tf-idf instead of the raw frequencies of occurrence of a
token in a given document is to scale down the impact of tokens that occur
very frequently in a given corpus and that are hence empirically less
informative than features that occur in a small fraction of the training
corpus.

The formula that is used to compute the tf-idf for a term t of a document d
in a document set is tf-idf(t, d) = tf(t, d) * idf(t), and the idf is
computed as idf(t) = log [ n / df(t) ] + 1 (if ``smooth_idf=False``), where
n is the total number of documents in the document set

In [39]:
sw = stopwords.words('english')
# cvr = CountVectorizer(
#     encoding='ISO-8859-1',
#     stop_words=sw,
#     ngram_range=[2,4],
#     min_df=4,max_df=50
#     )

In [71]:
y_n = sentiments[
                (sentiments[emotion] == 'Positive emotion') |
                (sentiments[emotion] == 'Negative emotion')
                ]

In [121]:
y_n[emotion].value_counts()

Positive emotion    2978
Negative emotion     570
Name: is_there_an_emotion_directed_at_a_brand_or_product, dtype: int64

In [115]:
# X = cvr.fit_transform(sentiments['tweet_text'])
# tokens = tokenize(sentiments['tweet_text'])
tokens = tokenize(y_n['tweet_text'])
# X = cvr.fit_transform(tokens)
# X
lemmatizer = WordNetLemmatizer()
lem = []
for doc in tokens:
    lem.append(' '.join([lemmatizer.lemmatize(token) for token in doc.split(' ')]))
tfidf = TfidfVectorizer(ngram_range=[1,4],max_df=0.5,min_df=10)
# print(tokens)
X = tfidf.fit_transform(tokens)

In [116]:
vect = pd.DataFrame(X.toarray(),columns=tfidf.get_feature_names())
vect.head()

,about,about the,about to,aclu,action,action link,actually,ad,after,again,...,you speak quot,you speak quot mark,you tweet,you tweet quot,you tweet quot is,your,your ipad,your iphone,yrs,zazzlesxsw
0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.348367,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.196329,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [29]:
PCA?

Init signature:
PCA(
    n_components=None,
    *,
    copy=True,
    whiten=False,
    svd_solver='auto',
    tol=0.0,
    iterated_power='auto',
    random_state=None,
)
Docstring:     
Principal component analysis (PCA).

Linear dimensionality reduction using Singular Value Decomposition of the
data to project it to a lower dimensional space. The input data is centered
but not scaled for each feature before applying the SVD.

It uses the LAPACK implementation of the full SVD or a randomized truncated
SVD by the method of Halko et al. 2009, depending on the shape of the input
data and the number of components to extract.

It can also use the scipy.sparse.linalg ARPACK implementation of the
truncated SVD.

Notice that this class does not support sparse input. See
:class:`TruncatedSVD` for an alternative with sparse data.

Read more in the :ref:`User Guide <PCA>`.

Parameters
----------
n_components : int, float, None or str
    Number of components to keep.
    if n_components is not 

In [117]:
pca = PCA(n_components=0.85)

In [31]:
emotion  = 'is_there_an_emotion_directed_at_a_brand_or_product'
directed = 'emotion_in_tweet_is_directed_at'

In [118]:
pca.fit_transform(vect,y_n[emotion])

array([[-0.02828757, -0.03904504, -0.05439692, ...,  0.04031826,
         0.04961264,  0.00352633],
       [-0.03091112, -0.11160592, -0.09518222, ..., -0.02816939,
        -0.02183693, -0.02596585],
       [-0.03269821, -0.01290945, -0.01433204, ...,  0.00458921,
         0.03887284,  0.02908574],
       ...,
       [ 0.04327187, -0.11066595,  0.20276061, ...,  0.06856005,
        -0.04172619,  0.00923729],
       [-0.04099579, -0.09701641, -0.10548752, ...,  0.0186417 ,
         0.01613704,  0.02167043],
       [-0.018109  ,  0.06210292, -0.06842777, ...,  0.01494416,
        -0.03127936,  0.042141  ]])

In [119]:
pca.explained_variance_ratio_.cumsum()

array([0.01578003, 0.02927832, 0.04040168, 0.05044328, 0.059613  ,
       0.06852648, 0.07684327, 0.08469489, 0.09214256, 0.09922066,
       0.10595196, 0.11239323, 0.11857932, 0.12449319, 0.13020882,
       0.13586541, 0.14130069, 0.14656404, 0.15173897, 0.15685874,
       0.16180026, 0.16669367, 0.17144812, 0.17607212, 0.1806048 ,
       0.18505464, 0.18949031, 0.19378023, 0.19799648, 0.20214138,
       0.20623384, 0.2102634 , 0.21426768, 0.21817088, 0.22201055,
       0.22584845, 0.22963628, 0.23333933, 0.2369358 , 0.24051295,
       0.24404324, 0.24753798, 0.25093855, 0.25429719, 0.25762989,
       0.26092445, 0.26417073, 0.26740517, 0.27060985, 0.27381136,
       0.2769089 , 0.27998402, 0.28303414, 0.28607232, 0.28908969,
       0.29208116, 0.2950002 , 0.29790107, 0.30078542, 0.30364204,
       0.30648674, 0.30930475, 0.31209992, 0.31488185, 0.31764932,
       0.32040468, 0.32310006, 0.32577996, 0.32843086, 0.33107027,
       0.33368625, 0.33629375, 0.33889492, 0.34146573, 0.34402

In [120]:
len(pca.components_)

559

In [79]:
sentiments.head(10)

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,iPhone,Negative emotion
1,@jessedee Know about @fludapp ? Awesome iPad/i...,iPad or iPhone App,Positive emotion
2,@swonderlin Can not wait for #iPad 2 also. The...,iPad,Positive emotion
3,@sxsw I hope this year's festival isn't as cra...,iPad or iPhone App,Negative emotion
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Google,Positive emotion
5,@teachntech00 New iPad Apps For #SpeechTherapy...,NaN,No emotion toward brand or product
7,"#SXSW is just starting, #CTIA is around the co...",Android,Positive emotion
8,Beautifully smart and simple idea RT @madebyma...,iPad or iPhone App,Positive emotion
9,Counting down the days to #sxsw plus strong Ca...,Apple,Positive emotion
10,Excited to meet the @samsungmobileus at #sxsw ...,Android,Positive emotion


In [80]:
sentiments.sample(5)['tweet_text']

1694    @mention #qagb with @mention Listening to @men...
1376    Apple pop-up shop for #SXSW. How cool is that?...
518     Thank you for these! RT @mention Public notes ...
6242    RT @mention Leaving Austin and I sort of want ...
8881    I'll be tweeting out #SXSW sessions I'm going ...
Name: tweet_text, dtype: object

In [81]:
sentiments.iloc[5014]['tweet_text']

'save me some cash! TechCrunch Giveaway: An iPad 2åÊ#TechCrunch {link} via @mention #winning! #ipad2 #sxsw #apple'

In [82]:
for col in ['emotion_in_tweet_is_directed_at','is_there_an_emotion_directed_at_a_brand_or_product']:
    display(sentiments[col].value_counts())
sentiments.info()

iPad                               946
Apple                              661
iPad or iPhone App                 470
Google                             430
iPhone                             297
Other Google product or service    293
Android App                         81
Android                             78
Other Apple product or service      35
Name: emotion_in_tweet_is_directed_at, dtype: int64

No emotion toward brand or product    5388
Positive emotion                      2978
Negative emotion                       570
I can't tell                           156
Name: is_there_an_emotion_directed_at_a_brand_or_product, dtype: int64

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9092 entries, 0 to 9092
Data columns (total 3 columns):
 #   Column                                              Non-Null Count  Dtype 
---  ------                                              --------------  ----- 
 0   tweet_text                                          9092 non-null   object
 1   emotion_in_tweet_is_directed_at                     3291 non-null   object
 2   is_there_an_emotion_directed_at_a_brand_or_product  9092 non-null   object
dtypes: object(3)
memory usage: 284.1+ KB


In [83]:
# sentiments.isna()
vals_ = sentiments[sentiments['emotion_in_tweet_is_directed_at'].isna()].value_counts('is_there_an_emotion_directed_at_a_brand_or_product')
vals_

is_there_an_emotion_directed_at_a_brand_or_product
No emotion toward brand or product    5297
Positive emotion                       306
I can't tell                           147
Negative emotion                        51
dtype: int64

In [84]:
sentiments[sentiments['tweet_text'].isna()]

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product


In [85]:
vals_.values.sum()

5801

In [86]:
vect.head()

,about,about the,about to,aclu,action,action link,actually,ad,after,again,...,you speak quot,you speak quot mark,you tweet,you tweet quot,you tweet quot is,your,your ipad,your iphone,yrs,zazzlesxsw
0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.348367,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.196329,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
